# NasNet Dog breed classification

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *
PATH = "data/dogbreed/"
sz=224; bs=32

In [3]:
def nasnet(pre): return nasnetalarge(pretrained = 'imagenet' if pre else None)
model_features[nasnet]=4032*2

In [4]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

In [ ]:
stats = ([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                    val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

In [ ]:
learn = ConvLearner.pretrained(nasnet, data, precompute=True, xtra_fc=[], ps=0.8)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
lr = 1e-2

In [ ]:
%time learn.fit(lr, 3)

In [ ]:
learn

In [ ]:
learn.precompute=False
learn.bn_freeze=True

In [ ]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('nas_pre')

In [ ]:
def get_data(sz, bs):
    stats = ([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                        val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

In [ ]:
sz = 400

In [ ]:
learn.set_data(get_data(sz, bs))
learn.freeze()

In [ ]:
learn.fit(lr, 3, cycle_len=1)

In [ ]:
learn.precompute=False
learn.bn_freeze=True

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA(is_test=True)
probs = np.exp(log_preds)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}results/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}submission-nasnet.csv', index=False)